In [3]:
from constants import *
from dotenv import load_dotenv
import os
import subprocess
import json
from pprint import pprint
from web3 import Web3
from bit import wif_to_key
from web3.middleware import geth_poa_middleware
from eth_account import Account

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

#mnemonic = "ramp stomach hotel sun craft traffic huge cage auction inspire narrow produce shock boss surge"

mnemonic = os.getenv('MNEMONIC', 'ramp stomach hotel sun craft traffic huge cage auction inspire narrow produce shock boss surge')

def derive_wallets(coin, Numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --coin={coin} --numderive={Numderive} --cols=path,address,privkey,pubkey --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

print(derive_wallets(BTC, 3))
coins = {ETH: derive_wallets(ETH, 3), BTCTEST: derive_wallets(BTCTEST, 3)}
pprint(coins)

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin ==BTCTEST:
        return PrivateKeyTestnet(priv_key)

def create_tx(coin, account, address_to, amount):
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account,
             "to": address_to,
             "value": amount}
        )
        return {
        'from': account.address,
        'to': address_to,     
        'value': amount, 
        'gas': w3.eth.gasPrice, 
        'gasPrice': gasEstimate, 
        'nonce': w3.eth.getTransactionCount(account.address),
        'chainID': 2424}
    if coin ==BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

def send_tx(coin, account, to, amount):
    if coin ==ETH:
        raw_tx=create_tx(coin, account, to, amount)
        signed= account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin ==BTCTEST:
        raw_tx=create_tx(coin, account, to, amount)
        signed= account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

[{'path': "m/44'/0'/0'/0/0", 'address': '1A8KcHcji6YnfYwEJZQLzcAn9NMYQR9jdK', 'privkey': 'L4vxUrXrcLpt34fPgVatyUCtoyd4sGjpC3DP8ggf8LZLopPoKH47', 'pubkey': '02912c4c8c74bc1e629cee4ad89c14a25b2f18ac89073eefb3b0d53c93805bcfe4'}, {'path': "m/44'/0'/0'/0/1", 'address': '1KarLCVVieRyn8Z1WLEkNqwMa8dq1xtm77', 'privkey': 'L5fmHf7vh34jPrc3gkfQeAg3pfAjK3JJDijk1GWzcD5B1cTyatAG', 'pubkey': '02cc2a6355bb028f9c9ed40cbb8212bd8b32d77368f915b8fe3615980f9cdf20d1'}, {'path': "m/44'/0'/0'/0/2", 'address': '1ECVkBLoGLWAggZ6jKB1obDYd5Y7M8RXQD', 'privkey': 'L25jgtxQxU4BCXX2miKv3kk546EBk1TWcyv5RPMuuZLWctDf2oTt', 'pubkey': '03ef675f292e488f6813fb4643c5b4f9cf9142a07bcb63235293b847f613d8bffc'}]
{'btc-test': [{'address': 'miLvzXGQhLriDpamjMGysMBNdgN6Lc3xxh',
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cVJFcCsEaV13VfcbxeWD3zoDenqzVW7oGQkJHWK5tro88zsE9iwz',
               'pubkey': '028e31f029c7daa57b839646bd61684a47a4c148da75a58ec8c6e7e487ca47f62b'},
              {'address': 'mvKKAkXMSm39n